# Starter Notebook

Install and import required libraries

In [1]:
# !pip install transformers datasets evaluate accelerate peft trl bitsandbytes
# !pip install nvidia-ml-py3

In [2]:
import os
import pandas as pd
import torch
import random
import re
import nltk
import pickle
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac

from transformers import (
    RobertaModel,
    RobertaTokenizer, 
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    RobertaForSequenceClassification,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel

# Download required NLTK data
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/viewsetting/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/viewsetting/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# 设置设备
device = torch.device('cuda:1' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"使用设备: {device}")

使用设备: cuda:1


In [4]:
import wandb
wandb.login(key="2008ab8d896bfc68619ace7f820e0513468b9783", relogin=True)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/viewsetting/.netrc


True

In [5]:
# Get current wandb entity
current_entity = wandb.api.default_entity
print(f"Current wandb entity: {current_entity}")


Current wandb entity: jl10897-new-york-university


## Load Tokenizer and Preprocess Data

In [6]:
# base_model = 'roberta-base'
# # base_model = 'roberta-large'

# dataset = load_dataset('ag_news', split='train')
# tokenizer = RobertaTokenizer.from_pretrained(base_model)

# def clean_text(text):
#     text = re.sub(r"<.*?>", "", text)                 # 去除 HTML 标签
#     text = re.sub(r"http\S+|www\S+", "", text)        # 移除 URL
#     text = re.sub(r"[^A-Za-z0-9.,!?;:'\"()\[\]\s]", "", text)  # 去除特殊字符
#     text = re.sub(r"\s+", " ", text).strip()          # 去除多余空格
#     return text.lower()                               # 可选：统一小写
    
# def preprocess(examples):
#     cleaned_texts = [clean_text(t) for t in examples['text']]
#     return tokenizer(cleaned_texts, truncation=True, padding=True, max_length=256)

# tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
# tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [7]:
nltk.download('averaged_perceptron_tagger_eng', download_dir='/home/viewsetting/nltk_data')



[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/viewsetting/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [8]:
nltk.data.path.append('/home/viewsetting/nltk_data')

In [9]:
base_model = 'roberta-base'
# base_model = 'roberta-large'

dataset = load_dataset('ag_news', split='train')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

# Initialize augmenters
synonym_aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)
delete_aug = naw.RandomWordAug(action='delete', aug_p=0.1)
swap_aug = naw.RandomWordAug(action='swap', aug_p=0.1)
typo_aug = nac.RandomCharAug(action='swap', aug_char_p=0.05, aug_word_p=0.1)

def clean_text(text):
    text = re.sub(r"<.*?>", "", text)                 # 去除 HTML 标签
    text = re.sub(r"http\S+|www\S+", "", text)        # 移除 URL
    text = re.sub(r"[^A-Za-z0-9.,!?;:'\"()\[\]\s]", "", text)  # 去除特殊字符
    text = re.sub(r"\s+", " ", text).strip()          # 去除多余空格
    return text.lower()                               # 统一小写

def simple_paraphrase(text):
    # Simple rule-based paraphrasing
    replacements = {
        'said': 'stated',
        'big': 'large',
        'small': 'tiny',
        'good': 'excellent',
        'bad': 'poor',
        'buy': 'purchase',
        'sell': 'trade',
        'make': 'create',
        'show': 'display',
        'start': 'begin'
    }
    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in replacements and random.random() < 0.3:
            words[i] = replacements[word.lower()]
    return ' '.join(words)

def preprocess_with_augmentation(examples, augmentations=None):
    if augmentations is None:
        augmentations = {
            'synonym': False,
            'delete': False,
            'swap': False,
            'paraphrase': False,
            'noise': False
        }

    # Clean texts first
    cleaned_texts = [clean_text(t) for t in examples['text']]
    augmented_texts = []

    for text in cleaned_texts:
        aug_text = text
        if augmentations.get('synonym', False):
            aug_text = synonym_aug.augment(aug_text)[0]
        if augmentations.get('delete', False):
            aug_text = delete_aug.augment(aug_text)[0]
        if augmentations.get('swap', False):
            aug_text = swap_aug.augment(aug_text)[0]
        if augmentations.get('paraphrase', False):
            aug_text = simple_paraphrase(aug_text)
        if augmentations.get('noise', False):
            aug_text = typo_aug.augment(aug_text)[0]
        augmented_texts.append(aug_text)

    # Tokenize with same parameters as original
    return tokenizer(augmented_texts, truncation=True, padding=True, max_length=256)

# Example augmentation configuration
augmentation_config = {
    'synonym': True,
    'delete': True,
    'swap': True,
    'paraphrase': True,
    'noise': True
}

tokenized_dataset = dataset.map(
    lambda examples: preprocess_with_augmentation(examples, augmentation_config),
    batched=True,
    remove_columns=["text"]
)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Parameter 'function'=<function <lambda> at 0x7646e553b8b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

In [10]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


## Load Pre-trained Model
Set up config for pretrained model and download it from hugging face

In [11]:
model = RobertaForSequenceClassification.from_pretrained(
    base_model,
    id2label=id2label)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

## Anything from here on can be modified

In [12]:
# Split the original training set
split_datasets = tokenized_dataset.train_test_split(test_size=640, seed=42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

## Setup LoRA Config
Setup PEFT config and get peft model for finetuning

In [13]:
# PEFT Config
# peft_config = LoraConfig(
#     r=2,
#     lora_alpha=4,
#     lora_dropout=0.05,
#     bias = 'none',
#     target_modules = ['query'],
#     task_type="SEQ_CLS",
# )
peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "value"],
    task_type="SEQ_CLS"
)

In [14]:
# !module load gcc
# !which gcc


In [15]:
# import os
# os.environ["CC"] = "/share/apps/NYUAD5/gcc/9.2.0/bin/gcc"


In [16]:
peft_model = get_peft_model(model, peft_config)
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): Module

In [17]:
# print("Trainable parameters:")
# for name, param in peft_model.named_parameters():
#     if param.requires_grad:
#         print(name)

In [18]:
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 741,124 || all params: 125,389,832 || trainable%: 0.5911


## Training Setup

In [19]:
# To track evaluation accuracy during training
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy
    }

In [23]:
import wandb
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Dict
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix,
    classification_report
)

def compute_metrics(pred):
    """
    Compute and log comprehensive evaluation metrics
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = torch.nn.functional.softmax(torch.tensor(pred.predictions), dim=-1)
    
    # Basic metrics
    accuracy = accuracy_score(labels, preds)
    precision_macro = precision_score(labels, preds, average='macro')
    precision_micro = precision_score(labels, preds, average='micro')
    precision_per_class = precision_score(labels, preds, average=None)
    
    recall_macro = recall_score(labels, preds, average='macro')
    recall_micro = recall_score(labels, preds, average='micro')
    recall_per_class = recall_score(labels, preds, average=None)
    
    f1_macro = f1_score(labels, preds, average='macro')
    f1_micro = f1_score(labels, preds, average='micro')
    f1_per_class = f1_score(labels, preds, average=None)
    
    # Per-class metrics
    metrics_per_class = {}
    for i, class_name in enumerate(id2label.values()):
        metrics_per_class.update({
            f'precision_class_{class_name}': precision_per_class[i],
            f'recall_class_{class_name}': recall_per_class[i],
            f'f1_class_{class_name}': f1_per_class[i]
        })
    
    # Confusion matrix - only log to wandb
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(id2label.values()),
                yticklabels=list(id2label.values()))
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Log confusion matrix to wandb
    wandb.log({'eval/confusion_matrix': wandb.Image(plt)})
    plt.close()
    
    # Confidence histograms - only log to wandb
    for i in range(len(id2label)):
        plt.figure(figsize=(8,6))
        class_probs = probs[:, i].numpy()
        plt.hist(class_probs, bins=50)
        plt.title(f'Confidence Distribution - Class {id2label[i]}')
        plt.xlabel('Confidence')
        plt.ylabel('Count')
        wandb.log({f'eval/confidence_dist_class_{id2label[i]}': wandb.Image(plt)})
        plt.close()

    # Log metrics to wandb
    wandb.log({
        'eval/accuracy': accuracy,
        'eval/precision_macro': precision_macro,
        'eval/precision_micro': precision_micro,
        'eval/recall_macro': recall_macro,
        'eval/recall_micro': recall_micro,
        'eval/f1_macro': f1_macro,
        'eval/f1_micro': f1_micro,
    })
    
    # Return only JSON-serializable metrics
    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_micro': precision_micro,
        'recall_macro': recall_macro,
        'recall_micro': recall_micro,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        **metrics_per_class
    }

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.gradient_norm = 0.0

    def training_step(self, model, inputs):
        """Override training step to compute training metrics"""
        loss = super().training_step(model, inputs)
        
        # Compute training accuracy
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = outputs.logits.argmax(-1)
            accuracy = (predictions == inputs['labels']).float().mean()
            
            # Log training metrics
            if self.state.global_step % self.args.logging_steps == 0:
                wandb.log({
                    'train/loss': loss.item(),
                    'train/accuracy': accuracy.item(),
                    'train/step': self.state.global_step
                })
        
        # Compute gradient norm
        if self.args.gradient_checkpointing:
            self.gradient_norm = torch.norm(
                torch.stack([
                    torch.norm(p.grad.detach())
                    for p in model.parameters()
                    if p.grad is not None
                ])
            ).item()
        
        return loss

    def log(self, logs: Dict[str, float]) -> None:
        """
        Enhanced logging with additional training metrics
        """
        if self.state.global_step % self.args.logging_steps == 0:
            # Log learning rates
            if hasattr(self.optimizer, "param_groups"):
                current_lr = self.optimizer.param_groups[0]['lr']
                wandb.log({
                    'train/learning_rate': current_lr,
                    'train/step': self.state.global_step
                })
            
            # Log gradient norm
            logs["train/gradient_norm"] = self.gradient_norm
            
            # Log batch size
            logs["train/batch_size"] = self.args.per_device_train_batch_size
            
            # Log memory usage if using GPU
            if torch.cuda.is_available():
                logs["system/gpu_memory_allocated"] = torch.cuda.memory_allocated() / 1024**2  # MB
                logs["system/gpu_memory_cached"] = torch.cuda.memory_reserved() / 1024**2  # MB
            
            # Log parameter statistics
            for name, param in self.model.named_parameters():
                if param.requires_grad:
                    logs[f"parameters/mean/{name}"] = param.data.mean().item()
                    logs[f"parameters/std/{name}"] = param.data.std().item()
                    if param.grad is not None:
                        logs[f"gradients/mean/{name}"] = param.grad.data.mean().item()
                        logs[f"gradients/std/{name}"] = param.grad.data.std().item()
            
            # Log all metrics to wandb
            wandb.log(logs)
        
        super().log(logs)

# Setup Training args
output_dir = "results"

training_args = TrainingArguments(
    output_dir=output_dir,
    report_to='wandb',
    eval_strategy='steps',
    logging_steps=100,
    eval_steps=200,
    save_steps=400,
    save_total_limit=2,

    learning_rate=1e-4,  # LoRA
    warmup_ratio=0.1,
    num_train_epochs=2,
    
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,

    optim="adamw_torch",  # BETTER THAN SGD
    weight_decay=0.01,

    gradient_checkpointing=False,
    dataloader_num_workers=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    run_name="lora_finetuning_run_3",
    
    logging_first_step=True, 
    logging_nan_inf_filter=False, 
    logging_strategy="steps",  
    label_names=["labels"]
)

def get_trainer(model):
    """
    Create a trainer instance with enhanced logging
    """
    # Initialize wandb with detailed config
    wandb.init(
        project="ag_news_classification",
        name=f"roberta_lora_{wandb.util.generate_id()}",
        config={
            "model_name": base_model,
            "lora_config": {
                "r": peft_config.r,
                "alpha": peft_config.lora_alpha,
                "dropout": peft_config.lora_dropout,
                "target_modules": peft_config.target_modules,
            },
            "training_config": {
                "learning_rate": training_args.learning_rate,
                "batch_size": training_args.per_device_train_batch_size,
                "epochs": training_args.num_train_epochs,
                "warmup_ratio": training_args.warmup_ratio,
                "weight_decay": training_args.weight_decay,
            },
            "augmentation_config": augmentation_config,
            "dataset": "ag_news",
            "train_size": len(train_dataset),
            "eval_size": len(eval_dataset),
        }
    )
    
    model.config.label2id = {label: i for i, label in enumerate(class_names)}
    model.config.id2label = {i: label for i, label in enumerate(class_names)}
    
    return CustomTrainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

### Start Training

In [24]:
!export CUDA_VISIBLE_DEVICES=3


In [25]:
peft_lora_finetuning_trainer = get_trainer(peft_model)

result = peft_lora_finetuning_trainer.train()

eval/accuracy,▁▁██
eval/f1_class_Business,▁█
eval/f1_class_Sci/Tech,▁█
eval/f1_class_Sports,▁█
eval/f1_class_World,▁█
eval/f1_macro,▁▁██
eval/f1_micro,▁▁██
eval/loss,█▁
eval/precision_class_Business,▁█
eval/precision_class_Sci/Tech,▁█
eval/precision_class_Sports,▁█


/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Micro,Recall Macro,Recall Micro,F1 Macro,F1 Micro,Precision Class World,Recall Class World,F1 Class World,Precision Class Sports,Recall Class Sports,F1 Class Sports,Precision Class Business,Recall Class Business,F1 Class Business,Precision Class Sci/tech,Recall Class Sci/tech,F1 Class Sci/tech,Norm,Size,Memory Allocated,Memory Cached,Model.model.roberta.encoder.layer.0.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.0.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.0.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.0.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.1.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.1.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.1.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.1.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.2.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.2.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.2.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.2.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.3.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.3.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.3.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.3.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.4.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.4.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.4.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.4.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.5.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.5.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.5.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.5.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.6.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.6.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.6.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.6.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.7.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.7.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.7.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.7.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.8.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.8.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.8.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.8.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.9.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.9.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.9.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.9.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.10.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.10.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.10.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.10.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.11.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.

/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:37: UserWa

In [26]:
# 保存模型
peft_model_path = os.path.join(output_dir, "peft_model")
peft_model.save_pretrained(peft_model_path)
# 保存tokenizer
tokenizer.save_pretrained(peft_model_path)
print(f"Model saved to {peft_model_path}")


Model saved to results/peft_model


## Evaluate Finetuned Model


### Performing Inference on Custom Input
Uncomment following functions for running inference on custom inputs

In [27]:
# def classify(model, tokenizer, text):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
#     output = model(**inputs)

#     prediction = output.logits.argmax(dim=-1).item()

#     print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
#     return id2label[prediction]

In [28]:
# classify( peft_model, tokenizer, "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
# classify( peft_model, tokenizer, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")

### Run Inference on eval_dataset

In [29]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=32, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    if labelled:
        metric = evaluate.load('accuracy')

    # Loop over the DataLoader
    for batch in tqdm(eval_dataloader):
        # Move each tensor in the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.append(predictions.cpu())

        if labelled:
            # Expecting that labels are provided under the "labels" key.
            references = batch["labels"]
            metric.add_batch(
                predictions=predictions.cpu().numpy(),
                references=references.cpu().numpy()
            )

    # Concatenate predictions from all batches
    all_predictions = torch.cat(all_predictions, dim=0)

    if labelled:
        eval_metric = metric.compute()
        print("Evaluation Metric:", eval_metric)
        return eval_metric, all_predictions
    else:
        return all_predictions

In [30]:
# Check evaluation accuracy
_, _ = evaluate_model(peft_model, eval_dataset, True, 32, data_collator)

100%|██████████| 20/20 [00:04<00:00,  4.19it/s]

Evaluation Metric: {'accuracy': 0.88125}


### Run Inference on unlabelled dataset

In [31]:
#Load your unlabelled data
unlabelled_dataset = pd.read_pickle("/home/viewsetting/ssd_2T/test_unlabelled.pkl")
test_dataset = unlabelled_dataset.map(preprocess_with_augmentation, batched=True, remove_columns=["text"])
unlabelled_dataset

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 8000
})

In [32]:
# Run inference and save predictions
preds = evaluate_model(peft_model, test_dataset, False, 32, data_collator)
df_output = pd.DataFrame({
    'ID': range(len(preds)),
    'Label': preds.numpy()  # or preds.tolist()
})
df_output.to_csv(os.path.join(output_dir,"inference_output.csv"), index=False)
print("Inference complete. Predictions saved to inference_output.csv")

100%|██████████| 250/250 [00:31<00:00,  7.94it/s]

Inference complete. Predictions saved to inference_output.csv
